# [Spinal Cord Toolbox (SCT)](https://spinalcordtoolbox.com/index.html)

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/overview.png" align="center" width="750px">

## Installation Instructions
The instructions below are for MacOS and Linux platforms. \
For more installation instructions, such as installing on Windows or Docker, check [here](https://spinalcordtoolbox.com/user_section/installation.html).

#### Run the following (starting with $) in your terminal
Skip if you already have SCT and FSLeyes installed.

In [ ]:
# First we'll clone the SCT repository from GitHub 
$git -C spinalcordtoolbox pull || git clone https://github.com/spinalcordtoolbox/spinalcordtoolbox.git spinalcordtoolbox

# Change our directory to the newly created directory
$cd spinalcordtoolbox

#start installation
$./install_sct -y

In [ ]:
# Install the following
!${SCT_DIR}/python/envs/venv_sct/bin/pip install matplotlib-inline
!${SCT_DIR}/python/envs/venv_sct/bin/pip install Ipython

Open a new terminal, and run the following to install FSLeyes.\
[FSLeyes](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLeyes) is a GUI image viewer. We'll need this to view our input/output images, and perform labelling when necessary


In [ ]:
# Run the following to install fsleyes
$source spinalcordtoolbox/python/etc/profile.d/conda.sh
$conda create -c conda-forge -p ~/fsleyes_env fsleyes -y

# Next let's make fsleyes available in the terminal without having to activate its environment
$sudo ln -s ~/fsleyes_env/bin/fsleyes /usr/local/bin/fsleyes

Test if SCT and FSLeyes are successfully installed, and working.

In [ ]:
# Run the following in the terminal
$sct_testing

# Run the following to check for fsleyes...
$fsleyes

In [ ]:
# library to open our results .html instead of from the terminal
import webbrowser

# [Gray matter segmentation - Tutorial](https://spinalcordtoolbox.com/user_section/tutorials/gray-matter-segmentation.html#gray-matter-segmentation)

### Gray and white matter segmentation

Without data there's no SCT, so let's download one for this tutorial by running the following.

In [ ]:
# If you're on linux run
!wget https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_gm-wm-segmentation.zip -O data_gm-wm-segmentation.zip -q

# Otherwise, if you're on MacOS run
# !curl -L https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_gm-wm-segmentation.zip -o data_gm-wm-segmentation.zip

!unzip -o data_gm-wm-segmentation.zip -d gandwm_seg && rm data_gm-wm-segmentation.zip

In [ ]:
%cd 'gandwm_seg/single_subject/data'
pwd = %pwd

#### Gray matter segmentation algorithm: **sct_deepseg_gm**
**sct_deepseg_gm** is restricted to T2*-like contrasts (GM bright, WM dark), but very effective and robust.

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/gm-wm-segmentation/io-sct_deepseg_gm.png" align="center" width="600px">

Input arguments:
* -i : Destination image the template will be warped to.
* -qc : Directory for Quality Control reporting.

Output:
* t2s_gmseg.nii.gz : The image file containing the gray matter segmentation.



In [ ]:
# Compute gray mmatter
!sct_deepseg_gm -i t2s/t2s.nii.gz -qc ./gm_seg/qc_singleSubj/t2s

In [ ]:
url = "file://" + pwd + "/gm_seg/qc_singleSubj/t2s/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t2s/t2s.nii.gz -cm greyscale -a 100.0 t2s/t2s_gmseg.nii.gz -cm red -a 70.0

#### White matter segmentation algorithm (inverse of gray matter):
Step 1️⃣: Compute full spinal cord segmentation (using **sct_deepseg_sc**). You may refer to the Segmentation Tutorial \
Step 2️⃣: subtract the gray matter from the full segmentation (using **sct_maths**).

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/gm-wm-segmentation/io-sct_maths_gm_wm.png" align="center" width="750px">

In [ ]:
# Step 1: Compute full spinal cord segmentation
!sct_deepseg_sc -i t2s/t2s.nii.gz -c t2s -ofolder ./wm_seg -qc ./wm_seg/qc_singleSubj/t2s

In [ ]:
url = "file://" + pwd + "/wm_seg/qc_singleSubj/t2s/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t2s/t2s.nii.gz -cm greyscale wm_seg/t2s_seg.nii.gz -cm red -a 70.0

In [ ]:
# Step 2: Subtract to obtain white matter
!sct_maths -i wm_seg/t2s_seg.nii.gz -sub t2s/t2s_gmseg.nii.gz -o wm_seg/t2s_wmseg.nii.gz

In [ ]:
!fsleyes wm_seg/t2s_wmseg.nii.gz

### Computing metrics using GM and WM segmentations

In [ ]:
%cd ../../../

In [ ]:
# If you're on linux run
!wget https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_gm-wm-metric-computation.zip -O data_gm-wm-metric-computation.zip -q

# Otherwise, if you're on MacOS run
# !curl -L https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_gm-wm-metric-computation.zip -o data_gm-wm-metric-computation.zip

!unzip -o data_gm-wm-metric-computation.zip -d gandwm_metric && rm data_gm-wm-metric-computation.zip

In [ ]:
%cd 'gandwm_metric/single_subject/data'
pwd = %pwd

#### Cross-Sectional Area (CSA) computation for gray and white matter: **sct_process_segmentation**

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/gm-wm-metric-computation/io-sct_process_segmentation.png" align="center" width="650px">

Input arguments:
* -i : input image.
* -o : output CSV file.
* -perslice : Set this option to 1 to turn on per-slice computation.
* -angle-corr: angle correction is applied to account for undesired orientation of the spinal cord. 0 means no correction, while 1 means apply angle correction. Check documention for details.

Output files/folders:
* csa_wm.csv and csa_gm.csv: Two CSV files containing shape metrics for both the white and gray matter segmentations.

In [ ]:
!sct_process_segmentation -i t2s/t2s_wmseg.nii.gz -o ./csa_wm.csv -perslice 1 -angle-corr 0

In [ ]:
# Open results and analyse
!open ./csa_wm.csv

In [ ]:
!sct_process_segmentation -i t2s/t2s_gmseg.nii.gz -o ./csa_gm.csv -perslice 1 -angle-corr 0

In [ ]:
# Open results and analyse
!open ./csa_gm.csv

#### Extracting intensity values for gray and white matter: **sct_extract_metric**
Input arguments:
* -i : input image.
* -f : File or folder used to pick out specific regions from the input image. 
* -method : We specify bin to binarize the mask, then use the binary mask to select the voxels for the metric extraction.
* -z : axial slice region to extract metrics for.
* -o : name of the output file.
* -append : Whether or not to append the results to the end of an existing file, rather than overwriting it.

In [ ]:
!sct_extract_metric -i t2s/t2s.nii.gz -f t2s/t2s_wmseg.nii.gz -method bin -z 2:12 -o ./t2s_value.csv

In [ ]:
!open ./t2s_value.csv

In [ ]:
!sct_extract_metric -i t2s/t2s.nii.gz -f t2s/t2s_gmseg.nii.gz -method bin -z 2:12 -o ./t2s_value.csv -append 1

In [ ]:
!open ./t2s_value.csv

### Improving registration results using white and gray matter segmentations

In [ ]:
%cd ../../../

In [ ]:
# If you're on linux run
!wget https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_improving-registration-with-gm-seg.zip -O data_improving-registration-with-gm-seg.zip -q

# Otherwise, if you're on MacOS run
# !curl -L https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_improving-registration-with-gm-seg.zip -o data_improving-registration-with-gm-seg.zip 

!unzip -o data_improving-registration-with-gm-seg.zip -d improve_regn && rm data_improving-registration-with-gm-seg.zip

In [ ]:
%cd "improve_regn/single_subject/data"
pwd = %pwd

### [GM-informed registration between the PAM50 template and T2* data](https://spinalcordtoolbox.com/user_section/tutorials/gray-matter-segmentation/improving-registration-with-gm-seg/gm-informed-t2s-template-registration.html#gm-informed-registration-between-the-pam50-template-and-t2-data)

Due to the nature of the T2s data, we cannot use the typical **sct_register_to_template** function.\
A fix is to:\
1️⃣: Perform vertical labeling\
2️⃣: Perform template registration\
using a different contrast for the same subject like T2. The above steps will provide all the necessities to move forward with the T2s registration.\
Yow may revisit the segmentation and registration tutorials if need be.

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/improving-registration-with-gm-seg/io-sct_register_multimodal-t2s.png" align="center" width="700px">

In [ ]:
!sct_register_multimodal -i "${SCT_DIR}/data/PAM50/template/PAM50_t2s.nii.gz" \
                        -iseg "${SCT_DIR}/data/PAM50/template/PAM50_wm.nii.gz" \
                        -d t2s/t2s.nii.gz \
                        -dseg t2s/t2s_wmseg.nii.gz \
                        -initwarp t2/warp_template2anat.nii.gz \
                        -initwarpinv t2/warp_anat2template.nii.gz \
                        -owarp warp_template2t2s.nii.gz \
                        -owarpinv warp_t2s2template.nii.gz \
                        -param step=1,type=seg,algo=rigid:step=2,type=seg,algo=bsplinesyn,slicewise=1,iter=3 \
                        -qc ./reg_t2s/qc_singleSubj

In [ ]:
url = "file://" + pwd + "/reg_t2s/qc_singleSubj/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes ${SCT_DIR}/data/PAM50/template/PAM50_t2s.nii.gz t2s_reg.nii.gz

In [ ]:
!fsleyes ./t2s/t2s.nii.gz PAM50_t2s_reg.nii.gz

#### Reusing the GM-informed warping field to improve MTI registration

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/improving-registration-with-gm-seg/io-sct_register_multimodal-mt.png" align="center" width="650px">

In [ ]:
!sct_register_multimodal -i "${SCT_DIR}/data/PAM50/template/PAM50_t2.nii.gz" \
                        -iseg "${SCT_DIR}/data/PAM50/template/PAM50_cord.nii.gz" \
                        -d mt/mt1.nii.gz \
                        -dseg mt/mt1_seg.nii.gz \
                        -param step=1,type=seg,algo=centermass:step=2,type=seg,algo=bsplinesyn,slicewise=1,iter=3 \
                        -m mt/mask_mt1.nii.gz \
                        -initwarp ./warp_template2t2s.nii.gz \
                        -owarp ./warp_template2mt.nii.gz \
                        -qc ./improved_reg_t2s/qc_singleSubj

In [ ]:
url = "file://" + pwd + "/improved_reg_t2s/qc_singleSubj/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes mt/mt1.nii.gz PAM50_t2_reg.nii.gz